# **PROYECTO FINAL | BIG DATA & ANALYTICS | MUNDIAL QATAR 2022**


<img src=https://sc2.elpais.com.uy/files/article_main_up/uploads/2019/09/03/5d6eb181e460e.jpeg>

## **Extract & Load Pipeline (Web Scraping | Google Cloud Storage | Big Query)** 

El presente programa, se encaga de extraer información de distintas páginas web, utilizando web scraping, generar archivos .CSV, almacenar los archivos en Gooogle Cloud Storage y cargar los datos en Bigquery.

### ***Instalar Librería lxml***

In [ ]:
pip install lxml

  Using cached lxml-4.9.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.4 MB)
Note: you may need to restart the kernel to use updated packages.


### ***Importar librerías***

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import ssl
import pandas as pd
ssl._create_default_https_context = ssl._create_unverified_context
import os
from google.cloud import storage
from google.cloud import bigquery


### ***Cliente para interactuar con la API de Google Cloud Storage***

In [ ]:
storage_client = storage.Client()

### ***Declaración de variables***

In [ ]:
#DECLARAR VARIABLES COPAS MUNDIALES
url_mundiales = "https://www.marca.com/futbol/mundial/palmares.html"
table = pd.read_html(url_mundiales)

#DECLARAR TABLAS BIGQUERY
table_id = "proyecto-final-bigdata.raw_data.copas_mundiales_csv"


### ***Crear Dataframe***

In [ ]:
df_mundiales = table[0]



### ***Generar archivos .CSV | Almacenar en Google Cloud Storage***

In [ ]:
df_mundiales.to_csv('gs://proyecto_final_bigdata_bucket/raw/web_scraping/mundial_Qatar_2022/copas_mundiales.csv',index=False,header=False)


In [ ]:
def load_table_uri_csv(table_id):

    # Construct a BigQuery client object.
    client = bigquery.Client(project="proyecto-final-bigdata")

    # TODO(developer): Set table_id to the ID of the table to create.
    table_id = "proyecto-final-bigdata.raw_data.copas_mundiales_csv"

    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("anio", "STRING"),
            bigquery.SchemaField("pais_campeon", "STRING"),
            bigquery.SchemaField("pais_subcampeon", "STRING"),
            bigquery.SchemaField("sede", "STRING"),
        ],
        skip_leading_rows=1,
        # The source format defaults to CSV, so the line below is optional.
        source_format=bigquery.SourceFormat.CSV,
    )
    uri = "gs://proyecto_final_bigdata_bucket/raw/web_scraping/mundial_Qatar_2022/copas_mundiales.csv"

    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)  # Make an API request.
    print("Loaded {} rows.".format(destination_table.num_rows))
    # [END bigquery_load_table_gcs_csv]

load_table_uri_csv(table_id)

Loaded 22 rows.
